<h1 style=\"text-align: center; font-size: 50px;\">  Text Generation with Neural Networks and Torch MLflow Integration</h1>

# Notebook Overview
- Start Execution
- User Constants
- Install and Import Libraries
- Configure Settings
- Verify Assets
- Logging Model to MLflow
- Fetching the Latest Model Version from MLflow
- Loading the Model and Running Inference

## Start Execution

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("register_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-07-16 14:20:21 - INFO - Notebook execution started.


## User Constants

In [3]:
INITIAL_WORD = 'Love '
SIZE = 100

## Install and Import Libraries

In [4]:
# Standard Library Imports
import warnings
from pathlib import Path

# Third-Party Libraries
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# MLflow for Experiment Tracking and Model Management
import mlflow
from mlflow.types.schema import Schema, ColSpec
from mlflow.models import ModelSignature

torch.manual_seed(0)

## Configure Settings

In [5]:
warnings.filterwarnings("ignore")

In [6]:
# ------------------------ Define global experiment and run names to be used throughout the notebook ------------------------
EXPERIMENT_SET = "RNN text generation"
RUN_NAME = "RNN Text Generation"
MODEL_NAME = "dict_torch_rnn_model"
TORCH_MODEL = "dict_torch_rnn_model.pt"
REGISTER_NAME = "Shakespeare_Model"
EXPERIMENT_NAME = "Shakespeare Text Generation"

# ------------------------ Paths ------------------------
DATA_PATH = "../data/shakespeare.txt"
MODEL_DECODER_PATH = "models/decoder.pt"
MODEL_ENCODER_PATH = "models/encoder.pt"
MODEL_PATH = 'models/dict_torch_rnn_model.pt'

## Verify Assets

In [7]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")
        
log_asset_status(
    asset_path=DATA_PATH,
    asset_name="Shakespeare text",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio."
)

log_asset_status(
    asset_path=MODEL_DECODER_PATH ,
    asset_name="Decoder model",
    success_message="",
    failure_message="Please check if model folder was properly downloaded in your project on AI Studio."
)

log_asset_status(
    asset_path=MODEL_ENCODER_PATH,
    asset_name="Encoder model",
    success_message="",
    failure_message="Please check if model folder was properly downloaded in your project on AI Studio."
)

log_asset_status(
    asset_path=MODEL_PATH,
    asset_name="Rnn model",
    success_message="",
    failure_message="Please check if model folder was properly downloaded in your project on AI Studio."
)

2025-07-16 14:20:23 - INFO - Shakespeare text is properly configured. 
2025-07-16 14:20:23 - INFO - Decoder model is properly configured. 
2025-07-16 14:20:23 - INFO - Encoder model is properly configured. 
2025-07-16 14:20:23 - INFO - Rnn model is properly configured. 


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Creating the LSTM Model

In [9]:
# Get Text Data
with open(DATA_PATH,'r',encoding='utf8') as f:
    text = f.read()

In [10]:
all_characters = set(text) # creates a set of unique characters found in the text

In [11]:
class CharModel(nn.Module):
    def __init__(self, decoder, encoder, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5, use_gpu=False):
        """Initializes CharModel

        Args:
            decoder: Assigns a unique integer to each character in a dictionary format
            encoder : Reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers.
            all_chars: Set of unique characters found in the text.
            num_hidden: Number of hidden layers. Defaults to 256.
            num_layers: Number of layers. Defaults to 4.
            drop_prob: Regularization technique to prevent overfitting. Defaults to 0.5.
            use_gpu: If the model uses GPU. Defaults to False.
        """
        try:
            super().__init__()
            self.drop_prob = drop_prob
            self.num_layers = num_layers
            self.num_hidden = num_hidden
            self.use_gpu = use_gpu
            
            self.all_chars = all_chars
            self.decoder = torch.load(decoder)
            self.encoder = torch.load(encoder)
            
            self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
            self.dropout = nn.Dropout(drop_prob)
            self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
            logger.info("CharModel initialized successfully")
    
        except Exception as e:
            logger.error(f"Error initializing CharModel: {str(e)}")
      
    
    def forward(self, x, hidden):
        """Implementation of the CharModel logic, in which, the input passes through every step of the arquiteture

        Args:
            x: Input tensor with shape (batch size and senquency length) containing character indices.
            hidden: Tuple containing the inicial hidden states of the CharModel each with shape (batch size and senquency length).

        Returns:
            final_out: Output tensor representing the predicted logits for each character in the sequence.
            hidden: Tuple containing the final hidden states of the CharModel.
        """
        try:
            lstm_output, hidden = self.lstm(x, hidden)       
            drop_output = self.dropout(lstm_output)
            drop_output = drop_output.contiguous().view(-1, self.num_hidden)
            final_out = self.fc_linear(drop_output)
            
            return final_out, hidden
        
        except Exception as e:
            logger.error(f"Error implementing CharModel logic: {str(e)}")
    
    
    def hidden_state(self, batch_size):
        """
        Initializes and returns the initial hidden state for a recurrent neural network (e.g., LSTM).

        This method creates zero-filled tensors for the hidden state (h_0) and cell state (c_0), 
        supporting GPU execution if `self.use_gpu` is set to True.

        Args:
            batch_size: The number of sequences in the input batch, used to determine the tensor dimensions.

        Returns:
            Tuple: A tuple containing the hidden state and cell state tensors 
            with shape (num_layers, batch_size, num_hidden). Returns None if an exception occurs, and logs the error.
        """
        try:
            if self.use_gpu:
                hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).to(device),
                        torch.zeros(self.num_layers,batch_size,self.num_hidden).to(device))
            else:
                hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                        torch.zeros(self.num_layers,batch_size,self.num_hidden))
            
            return hidden
        except Exception as e:
            logger.error(f"Error Initializing and returning the initial hidden state: {str(e)}")

## Logging Model to MLflow

In [ ]:
class RNNModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Loads the model and associated artifacts (encoder, decoder) into memory.

        Args:
            context: MLflow context containing paths to model artifacts.
        """
        try:
            self.model = CharModel(
                            all_chars=all_characters,
                            num_hidden=512,
                            num_layers=3,
                            drop_prob=0.5,
                            use_gpu=False,
                            decoder=context.artifacts['decoder'],
                            encoder=context.artifacts['encoder']
                                            
                        )

            self.model.load_state_dict(torch.load(context.artifacts['model_state_dict']))
            self.model.eval()
            logger.info("Loading context done successfully")
        except Exception as e:
            logger.error(f"Error loading context: {str(e)}")

    def one_hot_encoder(self, encoded_text, num_uni_chars):
        """
        Convert categorical data into a fixed-size vector of numerical values.

        Args:
            encoded_text: Batch of encoded text.
            num_uni_chars: Number of unique characters

        """
        try:
            one_hot = np.zeros((encoded_text.size, num_uni_chars))
            one_hot = one_hot.astype(np.float32)
            one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
            one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
            
            return one_hot
        
        except Exception as e:
            logger.error(f"Error converting categorical data: {str(e)}")

    def predict_next_char(self, char, hidden=None, k=3):
        """
        Predicts the next character given an input character and the current hidden state.

        This method encodes the input character, feeds it through the trained character-level 
        language model (e.g., LSTM), and samples from the top-k most probable characters 
        to determine the next one. It also returns the updated hidden state for sequential prediction.

        Args:
            char: The input character to start prediction from.
            hidden: Current hidden state of the model. Each tensor has shape (num_layers, batch_size, num_hidden).
                If None, a new hidden state should be initialized before calling this method.
            k: Number of top predictions to sample from.

        Returns:
            A tuple containing the predicted next character and the updated hidden state.
        """
        try:
            encoded_text = self.model.encoder[char]
            encoded_text = np.array([[encoded_text]])
            encoded_text = self.one_hot_encoder(encoded_text, len(self.model.all_chars))
            inputs = torch.from_numpy(encoded_text)
            inputs = inputs.cpu()
                
            hidden = tuple([state.data for state in hidden])
            lstm_out, hidden = self.model(inputs, hidden)    
            probs = F.softmax(lstm_out, dim=1).data
            probs = probs.cpu()

            
            probs, index_positions = probs.topk(k)        
            index_positions = index_positions.numpy().squeeze()
            probs = probs.numpy().flatten()
            probs = probs/probs.sum()
            char = np.random.choice(index_positions, p=probs)

            return self.model.decoder[char], hidden
        except Exception as e:
            logger.error(f"Error predicting next char: {str(e)}")

    def generate_text(self, seed, size, k=3):
        """
        Generates a sequence of text using the trained character-level language model.

        Starting from a seed string, this method uses the model to predict the next character
        one at a time, feeding each predicted character back into the model. It continues
        this process until the desired output length is reached.

        Args:
            seed: The initial sequence of characters used to start the text generation.
            size: The number of characters to generate after the seed.
            k: Number of top character predictions to consider for sampling at each step.

        Returns:
            The full generated text including the seed and the newly predicted characters.
        """
        try:
            self.model.cpu()
                
            self.model.eval()
            output_chars = [c for c in seed]
            hidden = self.model.hidden_state(1)
            
            for char in seed:
                char, hidden = self.predict_next_char(char, hidden, k=k)
        
            output_chars.append(char)
            for i in range(size):
                char, hidden = self.predict_next_char(output_chars[-1], hidden, k=k)
                output_chars.append(char)
                
            return ''.join(output_chars)
        
        except Exception as e:
            logger.error(f"Error generating text: {str(e)}")
        
    def predict(self, context, model_input):
        """
        Runs inference using the loaded model and input data.

        Args:
            context: The MLflow context object.
            model_input : A dictionary containing 'seed' and 'size' keys.

        Returns:
             The output from the model containing the predicted text.
        """
        try:
            initial_word = model_input['initial_word'][0]
            size = model_input['size'][0]
            output = self.generate_text(seed=initial_word, size=size)
            
            return output
        except Exception as e:
            logger.error(f"Error predicting text: {str(e)}")

    @classmethod
    def log_model(cls, model_state_dict, decoder, encoder, demo_folder="../demo", config_path="../configs/config.yaml"): 
        """
        Logs the model to MLflow, including artifacts, dependencies, and input/output signatures.

        Args:
            model_state_dict: Path where the model is saved before logging.
            decoder: Assigns a unique integer to each character in a dictionary format
            encoder : Reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers.
            demo_folder: Path to the folder containing the compiled demo UI. Defaults to "demo".
            config_path: Path to configuration file. Defaults to "../configs/config.yaml".
        """
        try:
            input_schema = Schema(
                [
                    ColSpec("string", "initial_word"),
                    ColSpec("long", "size")
                ]
            )

            output_schema = Schema(
                [
                    ColSpec("string", "generated_text")
                ]
            )
        
            signature = ModelSignature(inputs=input_schema, outputs=output_schema)
                
            requirements = [
                "torch",
                "numpy"
            ]
            mlflow.pyfunc.log_model(
                model_state_dict,
                python_model=cls(),
                artifacts={
                    "model_state_dict": model_state_dict, 
                    'decoder': MODEL_DECODER_PATH, 
                    'encoder': MODEL_ENCODER_PATH, 
                    "demo": demo_folder,
                    "config": config_path
                },
                signature=signature,
                pip_requirements=requirements
            )
            logger.info("Logging model to MLflow done successfully")

        except Exception as e:
            logger.error(f"Error logging model to MLflow: {str(e)}")

In [13]:
mlflow.set_tracking_uri('/phoenix/mlflow')
mlflow.set_experiment(experiment_name= EXPERIMENT_NAME)

<Experiment: artifact_location='/phoenix/mlflow/920820733064782143', creation_time=1752673235935, experiment_id='920820733064782143', last_update_time=1752673235935, lifecycle_stage='active', name='Shakespeare Text Generation', tags={}>

In [14]:
model_state_dict = MODEL_PATH
register_name = REGISTER_NAME 

In [15]:
with mlflow.start_run(run_name = RUN_NAME) as run:
    logger.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    RNNModel.log_model(model_state_dict, MODEL_DECODER_PATH, MODEL_ENCODER_PATH)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/{model_state_dict}", name=register_name)

2025-07-16 14:20:24 - INFO - Run's Artifact URI: /phoenix/mlflow/920820733064782143/aefc55591c5f4f59b7e7449733733827/artifacts


2025-07-16 14:20:25 - INFO - Logging model to MLflow done successfully
Registered model 'Shakespeare_Model' already exists. Creating a new version of this model...
Created version '2' of model 'Shakespeare_Model'.


## Fetching the Latest Model Version from MLflow

In [16]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions(register_name, stages=["None"])
latest_model_version = model_metadata[0].version
latest_model_version

2

## Loading the Model and Running Inference

In [17]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/{register_name}/{latest_model_version}")
print(model.predict({"initial_word": INITIAL_WORD, "size": SIZE}))

2025-07-16 14:20:25 - INFO - CharModel initialized successfully
2025-07-16 14:20:27 - INFO - Loading context done successfully


Love and the Duke of Wentmoreland, and so
    so strange to take the canners. The King and Sir John Caesar


In [18]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-07-16 14:20:28 - INFO - ⏱️ Total execution time: 0m 6.13s
2025-07-16 14:20:28 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using Z by HP AI Studio.